In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
import numpy as np
import requests
from talib import MACD, RSI, BBANDS
from datetime import datetime, timedelta, date
from IPython.display import display, HTML
%matplotlib inline
import json
import math
from dateutil import parser
from pytz import timezone
import sys

In [2]:
start_day_to_analyze = '2020-06-12'
end_day_to_analyze = '2020-06-13'

In [3]:
db_conn = create_engine("postgresql://momentum@localhost:5432/tradedb")

In [4]:
api = tradeapi.REST(base_url="https://api.alpaca.markets")
session = requests.session()

In [5]:
query = f'''
SELECT * 
FROM 
new_trades as t
WHERE 
    tstamp >= '{start_day_to_analyze}' and 
    tstamp < '{end_day_to_analyze}' 
ORDER BY symbol, tstamp
'''
trades =  pd.read_sql_query(query, db_conn)

In [6]:
trades

,trade_id,algo_run_id,symbol,operation,qty,price,indicators,client_time,tstamp,stop_price,target_price
0,4291,1501,AKR,buy,587,14.04,"{'avg': 13.7108, 'rsi': 66.92835012827148, 'ma...",2020-06-12 16:58:50.018734+00:00,2020-06-12 16:58:50.338789,13.88,14.38
1,4300,1501,AKR,sell,587,13.86,"{'avg': 13.7424, 'rsi': [33.77245516789455, 32...",2020-06-12 17:40:13.767376+00:00,2020-06-12 17:40:14.255580,13.88,14.38
2,4257,1499,APPS,buy,507,9.37,"{'avg': 9.2253, 'rsi': 69.26129728459809, 'mac...",2020-06-12 16:06:01.241826+00:00,2020-06-12 16:06:01.608347,9.19,9.73
3,4258,1499,APPS,buy,49,9.37,"{'avg': 9.2253, 'rsi': 69.26129728459809, 'mac...",2020-06-12 16:06:01.434860+00:00,2020-06-12 16:06:01.907010,9.19,9.73
4,4269,1499,APPS,sell,556,9.40,"{'avg': 9.2325, 'rsi': [70.66858257878165, 70....",2020-06-12 16:18:08.471864+00:00,2020-06-12 16:18:08.995891,9.19,9.73
...,...,...,...,...,...,...,...,...,...,...,...
189,4285,1498,VSLR,sell,1,9.19,"{'avg': 9.0281, 'rsi': [62.33843237806332, 63....",2020-06-12 16:51:28.585472+00:00,2020-06-12 16:51:29.028732,8.91,9.66
190,4316,1498,VSLR,buy,309,9.24,"{'avg': 9.0549, 'rsi': 67.38052319697144, 'mac...",2020-06-12 18:32:05.432360+00:00,2020-06-12 18:32:05.919229,8.92,9.88
191,4336,1504,VSLR,buy,309,9.24,"{'avg': 9.0549, 'rsi': 67.38052319697144, 'mac...",1498,2020-06-12 18:56:09.431308,8.92,9.88
192,4383,1504,VSLR,sell,154,9.27,"{'avg': 9.0777, 'rsi': [65.86939883064579, 67....",2020-06-12 19:38:13.181205+00:00,2020-06-12 19:38:13.564288,8.92,9.88


In [7]:
query = f'''
SELECT * 
FROM 
algo_run as t
WHERE 
    start_time >= '{start_day_to_analyze}' and 
    start_time < '{end_day_to_analyze}' 
ORDER BY algo_run_id
'''
algo_runs = pd.read_sql_query(query, db_conn)
algo_runs.set_index('algo_run_id', inplace=True)

In [8]:
algo_runs

,algo_name,algo_env,build_number,parameters,start_time,end_time,end_reason,batch_id
algo_run_id,,,,,,,,
1476,momentum_long,PAPER,v0.1-511-gc951f0f,{'DSN': 'postgresql://momentum@10.63.80.3/trad...,2020-06-12 13:52:00.490829,None,None,d806b006-0fa1-493f-b3df-3c2c9dd64905
1477,momentum_long,PAPER,v0.1-511-gc951f0f,{'DSN': 'postgresql://momentum@10.63.80.3/trad...,2020-06-12 13:52:02.342052,None,None,d806b006-0fa1-493f-b3df-3c2c9dd64905
1478,momentum_long,PAPER,v0.1-511-gc951f0f,{'DSN': 'postgresql://momentum@10.63.80.3/trad...,2020-06-12 13:52:04.533503,None,None,d806b006-0fa1-493f-b3df-3c2c9dd64905
1479,momentum_long,PAPER,v0.1-511-gc951f0f,{'DSN': 'postgresql://momentum@10.63.80.3/trad...,2020-06-12 13:52:06.585393,None,None,d806b006-0fa1-493f-b3df-3c2c9dd64905
1480,momentum_long,PAPER,v0.1-511-gc951f0f,{'DSN': 'postgresql://momentum@10.63.80.3/trad...,2020-06-12 13:52:10.031103,None,None,d806b006-0fa1-493f-b3df-3c2c9dd64905
1481,momentum_long,PAPER,v0.1-511-gc951f0f,{'DSN': 'postgresql://momentum@10.63.80.3/trad...,2020-06-12 13:52:12.395317,None,None,d806b006-0fa1-493f-b3df-3c2c9dd64905
1482,momentum_long,PAPER,v0.1-511-gc951f0f,{'DSN': 'postgresql://momentum@10.63.80.3/trad...,2020-06-12 13:52:20.244927,None,None,d806b006-0fa1-493f-b3df-3c2c9dd64905
1483,momentum_long,PAPER,v0.1-511-gc951f0f,{'DSN': 'postgresql://momentum@10.63.80.3/trad...,2020-06-12 14:07:42.350973,None,None,831585b5-6fe7-4480-9a63-f81ad93378b9
1484,momentum_long,PAPER,v0.1-511-gc951f0f,{'DSN': 'postgresql://momentum@10.63.80.3/trad...,2020-06-12 14:07:43.902630,None,None,831585b5-6fe7-4480-9a63-f81ad93378b9


In [9]:
trade_details = {}
batch = {}
for index, row in trades.iterrows():
    algo_run_id = row['algo_run_id']
    batch_id = algo_runs.loc[algo_run_id]['batch_id']
    symbol = row['symbol']
    time = row['tstamp']
    operation = row['operation']
    price = row['price']
    indicators = row['indicators']
    qty = row['qty'] 
    stop_price = row['stop_price']
    target_price = row['target_price']
    if algo_run_id not in trade_details:
        trade_details[algo_run_id] = {}
    if symbol not in trade_details[algo_run_id]:
        trade_details[algo_run_id][symbol] = []
    trade_details[algo_run_id][symbol].append((operation, time, price, qty, indicators, stop_price, target_price))
    if batch_id not in batch:
        batch[batch_id] = []
    if algo_run_id not in batch[batch_id]:
        #print(batch_id, algo_run_id)
        batch[batch_id].append(algo_run_id)

In [10]:
revenues = {}
how_was_my_day = {}

for batch_id, count in batch.items():
    revenues[batch_id] = []
    symbol_name = []
    counts = []
    for run_id in batch[batch_id]:
        
        symbols = trades.loc[trades['algo_run_id'] == run_id]['symbol'].value_counts()
        for symbol, count in symbols.items():
            revnue = 0
            for trade in trade_details[run_id][symbol]:
                delta = trade[2] * trade[3] * (1 if trade[0] == 'sell' or trade[0] == 'buy_short' else -1)
                revnue += delta

            revenues[batch_id].append(revnue)
            counts.append(count)
            symbol_name.append(symbol)


    d = {
        'symbol': symbol_name,
        'count': counts,
        'revenue': revenues[batch_id],
    }
    how_was_my_day[batch_id] = pd.DataFrame(data=d)

In [11]:
for batch_id, how_did_i_do in how_was_my_day.items():
    #display(HTML(algo_runs.loc[algo_runs['batch_id'] == batch_id].to_html()))
    display(HTML(how_did_i_do.round(2).sort_values(by=['revenue']).to_html()))
    print(f'TOTAL REVENUE: {pd.Series(revenues[batch_id]).sum().round(2)} $ for {batch_id}')


,symbol,count,revenue
6,CLS,1,-7161.00
0,CAMP,7,-5439.13
21,NOVA,3,-4615.62
19,CNCE,1,-4559.28
2,MBI,3,-4532.26
26,SSL,13,-4203.25
25,DSS,1,-3245.13
12,VSLR,13,-2849.14
11,DRRX,1,-1656.48
4,CBD,2,-119.52


TOTAL REVENUE: -39050.73 $ for 7118a934-cb66-4cab-b362-f97e9bf79149


,symbol,count,revenue
6,NOVA,1,-4632.72
0,CAMP,4,-670.30
3,DRRX,2,-78.54
1,DSS,2,-71.40
5,MBI,2,-49.44
10,VSLR,3,-4.68
8,RWT,12,15.44
4,CLS,13,18.20
9,SSL,11,19.25
7,CNCE,2,23.55


TOTAL REVENUE: -5401.76 $ for 8f8d99a6-38f5-42c6-8b86-005f5c7d03d9


,symbol,count,revenue
3,PGEN,2,-487.34
1,VSLR,3,-107.71
4,UNFI,2,-107.36
2,INSG,2,-100.36
0,IMRN,9,53.05


TOTAL REVENUE: -749.72 $ for 121a1799-7192-486a-a26b-3df192d93f7d


In [14]:
minute_history = {}
for batch_id, count in batch.items():
    for run_id in batch[batch_id]:
        symbols = trades.loc[trades['algo_run_id'] == run_id]['symbol'].value_counts()
        for symbol, count in symbols.items():
            if symbol not in minute_history:
                minute_history[symbol] = api.polygon.historic_agg_v2(
                    symbol, 
                    1, 
                    'minute',
                    _from = str((datetime.strptime(start_day_to_analyze, '%Y-%m-%d')-timedelta(days=10)).date()),
                    to=str((datetime.strptime(end_day_to_analyze, '%Y-%m-%d')+timedelta(days=1)).date())).df


In [ ]:
def grouper(iterable):
    prev = None
    group = []
    for item in iterable:
        
        if not prev or -0.02 <= float(item - prev) / prev <= 0.02:
            group.append(item)
        else:
            yield group
            group = [item]
        prev = item
    if group:
        yield group

def find_resistance(current_value, minute_history, now):
    """calculate next resistance"""
    now = pd.Timestamp(now) - timedelta(hours=4, minutes=1)
    #print(now)
    #print(minute_history)
    minute_history_index = minute_history["close"].index.get_loc(now, method='nearest')
    #print(f'index:{minute_history_index} data:{minute_history["close"][minute_history_index]}')
    #print(minute_history["close"][minute_history_index-10:minute_history_index])
    for back_track_min in range(120, len(minute_history.index), 60):
        #print("start")
        series = (
            minute_history["close"][-back_track_min:minute_history_index].dropna().between_time("9:30", "16:00").resample("5min").max()
        ).dropna()
        #print(series[-10:])
        # tlog(f"{symbol} find_resistances(): {len(series)}")

        diff = np.diff(series.values)
        #print(diff)
        high_index = np.where((diff[:-1] >= 0) & (diff[1:] <= 0))[0] + 1
        #print(high_index)
        #print(high_index)
        if len(high_index) > 0:
            local_maximas = sorted(
                [series[i] for i in high_index if series[i] >= current_value]
            )
            #print(f"local_maximas={local_maximas}")
            if len(local_maximas) > 0:
                #print([i for i in high_index if series[i] >= current_value])
                return local_maximas
            continue
            
            clusters = dict(enumerate(grouper(local_maximas), 1))
            #print(f"cluster={clusters}")
            resistances = []
            for key, cluster in clusters.items():
                if len(cluster) > 1:
                    resistances.append(round(sum(cluster) / len(cluster), 2))
            resistances = sorted(resistances)

            if len(resistances) > 0:
                return resistances


    return None

def find_support(current_value, minute_history, now):
    """calculate support"""
    now = pd.Timestamp(now) - timedelta(hours=4)
    minute_history_index = minute_history["close"].index.get_loc(now, method='nearest')
    for back_track_min in range(120, len(minute_history.index), 60):
        series = (minute_history["close"][-back_track_min:minute_history_index].resample("5min").min()).dropna()
        diff = np.diff(series.values)
        high_index = np.where((diff[:-1] <= 0) & (diff[1:] > 0))[0] + 1
        if len(high_index) > 0:
            local_maximas = sorted(
                [series[i] for i in high_index if series[i] <= current_value]
            )
            if len(local_maximas) > 0:
                return local_maximas
            
    return None

In [ ]:
for batch_id, count in batch.items():
    for run_id in batch[batch_id]:
        for symbol in minute_history:    
            if symbol not in trade_details[run_id]:
                continue
            start_date = datetime.strptime(start_day_to_analyze, "%Y-%m-%d") + timedelta(hours=9, minutes=30)
            minute_history_index = minute_history[symbol]["close"].index.get_loc(start_date, method='nearest')
            plt.plot(minute_history[symbol]["close"][minute_history_index:].between_time("9:30", "16:00"), label=symbol)
            plt.xticks(rotation=45)
            delta = 0
            profit = 0

            operations = []
            deltas = []
            profits = []
            times = []
            prices = []
            qtys = []
            indicators = []
            target_price = []
            stop_price = []
            resistances = []
            supports = []
            for trade in trade_details[run_id][symbol]:
                resistance = None
                support = None
                if trade[0] == 'buy':
                    resistance = find_resistance(trade[2], minute_history[symbol], trade[1])

                    for r in resistance:
                        plt.axhline(y=r, color="r")
                    support = find_support(trade[2], minute_history[symbol], trade[1])
                    for s in support:
                        plt.axhline(y=s, color="g")  

                delta = trade[2] * trade[3] * (1 if trade[0] == 'sell' or trade[0] == 'buy_short' else -1)
                profit += delta
                plt.scatter(pd.to_datetime(trade[1]), trade[2], c='g' if trade[0] == 'buy' or trade[0] == 'sell_short' else 'r', s=100)
                deltas.append(round(delta, 2))
                profits.append(round(profit, 2))
                operations.append(trade[0])
                times.append(pd.to_datetime(trade[1]))
                prices.append(trade[2])
                qtys.append(trade[3])
                indicators.append(trade[4])
                target_price.append(trade[6])
                stop_price.append(trade[5])    
                resistances.append(resistance)
                supports.append(support)
            d = {
                'profit': profits,
                'trade': deltas,
                'operation': operations,
                'at': times,
                'price': prices,
                'support': supports,
                'resistance': resistances,
                'qty': qtys,
                'indicators': indicators,
                'target price': target_price,
                'stop price': stop_price
            }   
            print (f'{symbol} analysis for run {run_id} with profit {round(profit, 2)}')
            display(HTML(pd.DataFrame(data=d).to_html()))
            plt.legend()
            plt.show()

In [15]:
 pd.Timestamp('2020-04-16 13:44:06.066') 

Timestamp('2020-04-16 13:44:06.066000')

In [ ]:
index = minute_history['NURO'].index.get_loc(pd.Timestamp('2020-04-16 13:44:06.066') , method='nearest')

In [ ]:
minute_history['NURO'][:index]

In [ ]:
series = minute_history['NURO'][:2417]['close']

In [ ]:
MACD(series.dropna().between_time("9:30", "16:00"))

In [ ]:
for batch_id, count in batch.items():
    for run_id in batch[batch_id]:
        for symbol in minute_history:    
            if symbol not in trade_details[run_id]:
                continue
            start_date = datetime.strptime(start_day_to_analyze, "%Y-%m-%d") + timedelta(hours=9, minutes=30)
            minute_history_index = minute_history[symbol]["close"].index.get_loc(start_date, method='nearest')
            plt.plot(minute_history[symbol]["close"][minute_history_index:].between_time("9:30", "16:00"), label=symbol)
            plt.xticks(rotation=45)
            delta = 0
            profit = 0

            operations = []
            deltas = []
            profits = []
            times = []
            prices = []
            qtys = []
            indicators = []
            target_price = []
            stop_price = []
            resistances = []
            supports = []
            for trade in trade_details[run_id][symbol]:
                resistance = None
                support = None
                if trade[0] == 'buy':
                    resistance = find_resistance(trade[2], minute_history[symbol], trade[1])

                    for r in resistance:
                        plt.axhline(y=r, color="r")
                    support = find_support(trade[2], minute_history[symbol], trade[1])
                    for s in support:
                        plt.axhline(y=s, color="g")  

                delta = trade[2] * trade[3] * (1 if trade[0] == 'sell' or trade[0] == 'buy_short' else -1)
                profit += delta
                plt.scatter(pd.to_datetime(trade[1]), trade[2], c='g' if trade[0] == 'buy' or trade[0] == 'sell_short' else 'r', s=100)
                deltas.append(round(delta, 2))
                profits.append(round(profit, 2))
                operations.append(trade[0])
                times.append(pd.to_datetime(trade[1]))
                prices.append(trade[2])
                qtys.append(trade[3])
                indicators.append(trade[4])
                target_price.append(trade[6])
                stop_price.append(trade[5])    
                resistances.append(resistance)
                supports.append(support)
            d = {
                'profit': profits,
                'trade': deltas,
                'operation': operations,
                'at': times,
                'price': prices,
                'support': supports,
                'resistance': resistances,
                'qty': qtys,
                'indicators': indicators,
                'target price': target_price,
                'stop price': stop_price
            }   
            print (f'{symbol} analysis for run {run_id} with profit {round(profit, 2)}')
            display(HTML(pd.DataFrame(data=d).to_html()))
            plt.legend()
            plt.show()